In [1]:
from comet_ml import Experiment
experiment = Experiment(api_key="xktj4EX0zB8YcQ3BEaFwOQYpu")

COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable
COMET INFO: old comet version (1.0.9) detected. current: 1.0.10 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/incnas/general/dd361dd4943b4df99317fc8c16dcfcdd



In [2]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.regularizers import L1L2
from keras.utils import np_utils 

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
import fnmatch

import pandas as pd

import sklearn.preprocessing as preprocessing

## Load Data

In [4]:
train_path = "I:\\Honours-Project\\data\\sorted\\agg\\train\\"
train_csv = "I:\\Honours-Project\\data\\sorted\\agg\\train\\ids.csv"
test_path = "I:\\Honours-Project\\data\\sorted\\agg\\test\\"
test_csv = "I:\\Honours-Project\\data\\sorted\\agg\\test\\ids.csv"

dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"

csv = "I:\\Honours-Project\\data\\sorted\\agg\\ids.csv"

In [5]:
df = pd.read_csv(csv,index_col=0).sample(frac=1)

x = df.drop(columns=['name','date','flow_rate','source','moments','inertia_tensor','cluster'])
x = preprocessing.Normalizer().fit_transform(x)

In [6]:
test_size = int(df.shape[0]/10)

x_test = np.array(x[:test_size])
y_test = df['cluster'][:test_size].values

x_train = np.array(x[test_size:])
y_train = df['cluster'][test_size:].values

## Run Logistic Regression

In [7]:
batch_size = 16
nb_classes = 3
nb_epoch = 20

In [8]:
input_dim = x_train.shape[1]

In [9]:
y_train[y_train == 1800] = 0
y_train[y_train == 3600] = 1
y_train[y_train == 7200] = 2

y_test[y_test == 1800] = 0
y_test[y_test == 3600] = 1
y_test[y_test == 7200] = 2

y_train = np_utils.to_categorical(y_train, nb_classes) 
y_test = np_utils.to_categorical(y_test, nb_classes)

In [10]:
def build_logistic_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

    return model

In [11]:
model = build_logistic_model(input_dim, nb_classes)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [13]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 362 samples, validate on 40 samples
Epoch 1/20
362/362 [==============================] - 2s 6ms/step - loss: 0.9484 - acc: 0.5663 - val_loss: 0.9205 - val_acc: 0.5750
Epoch 2/20
362/362 [==============================] - 0s 362us/step - loss: 0.9339 - acc: 0.5663 - val_loss: 0.9058 - val_acc: 0.5750
Epoch 3/20
362/362 [==============================] - 0s 283us/step - loss: 0.9224 - acc: 0.5663 - val_loss: 0.8934 - val_acc: 0.5750
Epoch 4/20
362/362 [==============================] - 0s 282us/step - loss: 0.9126 - acc: 0.5663 - val_loss: 0.8832 - val_acc: 0.5750
Epoch 5/20
362/362 [==============================] - 0s 290us/step - loss: 0.9047 - acc: 0.5663 - val_loss: 0.8744 - val_acc: 0.5750
Epoch 6/20
362/362 [==============================] - 0s 348us/step - loss: 0.8979 - acc: 0.5663 - val_loss: 0.8673 - val_acc: 0.5750
Epoch 7/20
362/362 [==============================] - 0s 303us/step - loss: 0.8926 - acc: 0.5663 - val_loss: 0.8611 - val_acc: 0.5750
Epoch 8/20
362/362 

In [14]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.8243522882461548
Test accuracy: 0.575


In [15]:
weights, biases = model.layers[0].get_weights()
print('Weights: ', weights)
print('Bias: ', biases)

Weights:  [[-0.2796635  -0.34623784 -0.02587918]
 [-0.0282692  -0.15013832  0.34580246]
 [ 0.6791583  -0.90775836 -0.30607316]
 [ 0.4654623   0.5516795  -0.22086973]
 [-0.13993092 -0.33306053  0.5865878 ]
 [ 0.3498239   0.6021987   0.60364145]
 [-0.13396563 -0.43492132 -0.18710108]
 [ 0.5366579   0.4958366  -0.2729915 ]
 [-0.5391157   0.02592128  0.56895053]
 [ 0.4616033  -0.17874677 -0.22259244]]
Bias:  [-0.05134882 -0.22416243  0.27551123]
